In [6]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run, ir_datasets
from tira.rest_api_client import Client
import pyterrier as pt


In [21]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
input_dataset = 'ir-lab-sose-2024/ir-acl-anthology-20240504-training'
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', input_dataset)
# Get the topics
topics = pt_dataset.get_topics('query')

In [51]:
import gzip
import json
import pandas as pd


def create_transformer_from_json(json_file):
    # Load JSON data into DataFrame
    data = []
    with gzip.open(json_file, 'rt', encoding='utf-8') as file:
        for line in file:
            record = json.loads(line.strip())
            data.append(record)

    df = pd.DataFrame(data)
    df.rename(columns={'query_id': 'qid'}, inplace=True)

    # Create a transformer from the DataFrame
    transformer = pt.Transformer.from_df(df, uniform=True)
    return transformer

cot_prf_path = "llm-qe/cot_prf-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
cot_path = "llm-qe/cot-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2d_fs_msmarco_path = "llm-qe/q2d_fs_msmarco-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2d_fs_path =  "llm-qe/q2d_fs-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2d_prf_path = "llm-qe/q2d_prf-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2d_zs_path = "llm-qe/q2d_zs-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2e_fs_path = "llm-qe/q2e_fs-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2e_prf_path = "llm-qe/q2e_prf-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"
q2e_zs_path = "llm-qe/q2e_zs-gpt-4o-mini-ir-lab-sose-2024-ir-acl-anthology-20240504-training.jsonl.gz"

# Create a transformer from the JSON file
cot_prf = create_transformer_from_json(cot_prf_path)
cot = create_transformer_from_json(cot_path)
q2d_fs_msmarco = create_transformer_from_json(q2d_fs_msmarco_path)
q2d_fs = create_transformer_from_json(q2d_fs_path)
q2d_prf = create_transformer_from_json(q2d_prf_path)
q2d_zs = create_transformer_from_json(q2d_zs_path)
q2e_fs = create_transformer_from_json(q2e_fs_path)
q2e_prf = create_transformer_from_json(q2e_prf_path)
q2e_zs = create_transformer_from_json(q2e_zs_path)



In [62]:
tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

def pt_tokenize(text):
    return ' '.join(tokeniser.getTokens(text))

def expand_query(topic):
  ret = ' '.join([topic['query'],topic['query'],topic['query'],topic['query'],topic['query'],topic['query'],topic['query'],topic['query'],topic['query'],topic['query'], topic['llm_expansion']])

  # apply the tokenization
  return pt_tokenize(ret)

# we wrap this into an pyterrier transformer
# Documentation: https://pyterrier.readthedocs.io/en/latest/apply.html
pt_expand_query = pt.apply.query(expand_query)

In [53]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25_bo1 = bm25 >> pt.rewrite.Bo1QueryExpansion(index) >> bm25
bm25_rm3 = bm25 >> pt.rewrite.RM3(index) >> bm25
bm25_kl = bm25 >> pt.rewrite.KLQueryExpansion(index) >> bm25

In [63]:
pipeline_q2d_zs = (q2d_zs >> pt_expand_query) >> bm25
pipeline_q2d_fs = (q2d_fs >> pt_expand_query) >> bm25
pipeline_q2d_fs_msmarco = (q2d_fs_msmarco >> pt_expand_query) >> bm25
pipeline_q2d_prf = (q2d_prf >> pt_expand_query) >> bm25

pipeline_q2e_zs = (q2e_zs >> pt_expand_query) >> bm25
pipeline_q2e_fs = (q2e_fs >> pt_expand_query) >> bm25
pipeline_q2e_prf = (q2e_prf >> pt_expand_query) >> bm25

pipeline_cot = (cot >> pt_expand_query) >> bm25
pipeline_cot_prf = (cot_prf >> pt_expand_query) >> bm25

In [61]:
print("result with query*1, expansion*1")

pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*1, expansion*1


pt.Experiment:   0%|          | 0/13 [00:00<?, ?system/s]

pt.Experiment:  15%|█▌        | 2/13 [00:04<00:28,  2.62s/system]

13:59:58.765 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
13:59:58.908 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:58<00:00,  4.47s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.334285,0.557046,0.558333,0.824720
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.233468,0.500983,0.422583,0.716882
5,BM25+q2d_fs(training data),0.249561,0.536821,0.425362,0.737549
6,BM25+q2d_fs(msmarco),0.226902,0.451403,0.430762,0.717256
7,BM25+q2d_prf,0.295683,0.576255,0.475840,0.759188
8,BM25+q2e_zs,0.203412,0.386732,0.379809,0.691984
9,BM25+q2e_fs,0.236367,0.454178,0.454391,0.791643


In [57]:
print("result with query*5, expansion *1")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*5, expansion *1


pt.Experiment:   0%|          | 0/13 [00:00<?, ?system/s]

pt.Experiment:  15%|█▌        | 2/13 [00:05<00:35,  3.19s/system]

13:57:54.164 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
13:57:54.350 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:59<00:00,  4.57s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.334285,0.557046,0.558333,0.824720
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.298996,0.567174,0.517039,0.803946
5,BM25+q2d_fs(training data),0.327689,0.606379,0.523259,0.816709
6,BM25+q2d_fs(msmarco),0.319042,0.520409,0.545020,0.831599
7,BM25+q2d_prf,0.337850,0.607218,0.536623,0.835209
8,BM25+q2e_zs,0.330336,0.551438,0.530583,0.825605
9,BM25+q2e_fs,0.350054,0.553105,0.589487,0.854271


In [64]:
print("result with query*10, expansion *1")
pt.Experiment(
    [bm25, bm25_bo1, bm25_rm3, bm25_kl, pipeline_q2d_zs, pipeline_q2d_fs, pipeline_q2d_fs_msmarco, pipeline_q2d_prf, pipeline_q2e_zs, pipeline_q2e_fs, pipeline_q2e_prf, pipeline_cot, pipeline_cot_prf, ],
    names=['BM25', 'BM25+BO1', 'BM25+RM3', 'BM25+KL', 'BM25+q2d_zs', 'BM25+q2d_fs(training data)', 'BM25+q2d_fs(msmarco)', 'BM25+q2d_prf', 'BM25+q2e_zs', 'BM25+q2e_fs', 'BM25+q2e_prf', 'BM25+cot', 'BM25+cot_prf'],
    topics=topics,
    qrels=pt_dataset.get_qrels(),    
    eval_metrics=["ndcg_cut.10", "recip_rank", "recall_100", "recall_1000"],
    verbose=True
)

result with query*10, expansion *1


pt.Experiment:  15%|█▌        | 2/13 [00:04<00:29,  2.65s/system]

14:01:21.111 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
14:01:21.267 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


pt.Experiment: 100%|██████████| 13/13 [00:57<00:00,  4.41s/system]


,name,ndcg_cut.10,recip_rank,recall_100,recall_1000
0,BM25,0.374041,0.579877,0.601333,0.825376
1,BM25+BO1,0.378220,0.571535,0.595545,0.833643
2,BM25+RM3,0.334285,0.557046,0.558333,0.824720
3,BM25+KL,0.379452,0.565090,0.583871,0.831915
4,BM25+q2d_zs,0.351865,0.582197,0.570003,0.840642
5,BM25+q2d_fs(training data),0.369316,0.632794,0.594531,0.856184
6,BM25+q2d_fs(msmarco),0.370676,0.575057,0.594361,0.861028
7,BM25+q2d_prf,0.367564,0.625277,0.582286,0.847897
8,BM25+q2e_zs,0.381493,0.593133,0.576018,0.843157
9,BM25+q2e_fs,0.368586,0.586201,0.603162,0.849416
